In [1]:
# let's see the proto schema file
!cat my_proto.proto

syntax = "proto3";

message MyProtoMessage {
  string field1 = 1;
  int32 field2 = 2;
  // Add more fields as needed
}

In [2]:
# build the python module from the .proto file
# (nb: this requires using the protoc cli tool to generate a .py file)
!python3 -m pip install grpcio-tools
!python3 -m grpc_tools.protoc --version
!python3 -m grpc_tools.protoc -I=. --python_out=. my_proto.proto

libprotoc 25.1


In [3]:
# import the built module
from my_proto_pb2 import MyProtoMessage

In [4]:
# Example protobuf message instance
protobuf_message = MyProtoMessage(
    field1="value1",
    field2=123,
    # Add more fields as needed
)

In [5]:
from google.protobuf.json_format import MessageToJson, MessageToDict

In [6]:
from pydantic import BaseModel, ValidationError

In [7]:
# Define Pydantic model
class MyModel(BaseModel):
    class Config:
        strict = True
    field1: str
    field2: int

### Method 1: convert through JSON

In [8]:
# Convert protobuf message to json
protobuf_json = MessageToJson(protobuf_message)

In [9]:
print(protobuf_json)

{
  "field1": "value1",
  "field2": 123
}


In [10]:
try:
    # Validate incoming data
    json_validated_data = MyModel.parse_raw(protobuf_json)
    print("Data is valid:", json_validated_data)
except ValidationError as e:
    print("Validation error:", e)
    # Handle validation error based on your application's logic

Data is valid: field1='value1' field2=123


### Method 2: convert through python dict

In [11]:
# Convert protobuf message to json
protobuf_dict = MessageToDict(protobuf_message)

In [12]:
print(type(protobuf_dict))
print(protobuf_dict)

<class 'dict'>
{'field1': 'value1', 'field2': 123}


In [13]:
try:
    # Validate incoming data
    dict_validated_data = MyModel(**protobuf_dict)
    print("Data is valid:", dict_validated_data)
    dict_validated_data_2 = MyModel.parse_obj(protobuf_dict) # this way may be preferred, but first way could be more performant
    print("Data 2 is valid:",dict_validated_data_2)
except ValidationError as e:
    print("Validation error:", e)
    # Handle validation error based on your application's logic

Data is valid: field1='value1' field2=123
Data 2 is valid: field1='value1' field2=123
